In [1]:
# save model using the dill library instead of the joblib library, 
# make sure this import is on top as it will load the imported libraries as well
import dill

In [2]:
from sklearn.ensemble import IsolationForest
X = [[-1.1], [0.3], [0.5], [100]]
clf = IsolationForest(random_state=0).fit(X)
X_pred = [[0.1], [0], [90]]
clf.predict(X_pred)

array([ 1,  1, -1])

In [3]:
clf.decision_function(X_pred)

array([ 0.16355707,  0.14809923, -0.12865009])

In [4]:
class DecisionFunctionPredictor:

    def __init__(self, model):
        self.model = model

    def fit(self, X):
        self.model.fit(X)
        return self

    def predict(self, X):
        return self.model.predict(X)
    
    def predict_proba(self, X):
        return self.model.decision_function(X)

In [5]:
clf = DecisionFunctionPredictor(IsolationForest(random_state=0))
clf.fit(X)
clf.predict(X)

array([ 1,  1,  1, -1])

In [6]:
clf.predict_proba(X_pred)

array([ 0.16355707,  0.14809923, -0.12865009])

In [7]:
save_path = 'model.joblib'
with open(save_path, 'wb') as model_file:
    dill.settings['recurse']=True
    dill.dump(clf, model_file)

In [8]:
from zipfile import ZipFile
from io import BytesIO
import requests
from requests.auth import HTTPBasicAuth
def upload_model(path, model_name, description, upload_url, api_key, api_secret):
    
    zipfile_name = 'model.zip'
    # Create a zip
    with ZipFile('model.zip', 'w') as zipfile:
       # Add multiple files to the zip
       zipfile.write(path, 'model.joblib')
        
    with open('model.zip', 'rb') as f:
        upload_file = BytesIO(f.read())
        
    # upload to waylay
    resp = requests.post(upload_url,
                         files={"file": ('model.zip', upload_file)},
                         data={"name": model_name, "framework": "sklearn", "description": description},
                         auth=HTTPBasicAuth(api_key, api_secret))

    return resp.json()

In [9]:
api_key = 'your-api-key'
api_secret = 'your-api-secret'
byom_url = 'https://byoml.waylay.io/models'
model_name = 'decision-function'
description = 'Custom sklearn model example'
print(
    upload_model(
        save_path,
        model_name,
        description,
        byom_url,  
        api_key, 
        api_secret
    )
)

{'message': 'Model successfully uploaded'}


In [10]:
response = requests.post(byom_url + '/' + model_name + '/predict',
                         json = {"instances": X_pred},
                         auth=HTTPBasicAuth(api_key, api_secret)).json()
response

{'predictions': [1, 1, -1]}

In [11]:
response = requests.post(byom_url + '/' + model_name + '/predict',
                         json = {"instances": X_pred, "probabilities": True},
                         auth=HTTPBasicAuth(api_key, api_secret)).json()
response

{'predictions': [0.16355706941457837,
  0.14809923156656285,
  -0.12865009356674928]}

In [12]:
# check they are equal
response['predictions']==clf.predict_proba(X_pred)

array([ True,  True,  True])

In [13]:
response = requests.delete(byom_url + '/' + model_name,
                auth=HTTPBasicAuth(api_key, api_secret)).json()
response

{'message': 'Model successfully deleted'}